# Chapter 3: Computational linear algebra

**Before Class**:
* Read Chapter 3 in Savov (2020) and take notes
* Watch the following videos and take notes:
  * [Matrix multiplication as composition](https://www.3blue1brown.com/lessons/matrix-multiplication)
  * [Three-dimensional linear transformations](https://www.3blue1brown.com/lessons/3d-transformations)
  * [The determinant](https://www.3blue1brown.com/lessons/determinant)
  * [Inverse matrices, column space and null space](https://www.3blue1brown.com/lessons/inverse-matrices)
* Compile a list of questions to bring to class

**During and After Class**:
* Take notes (on paper or a tablet computer)
* Complete this notebook, submit you answer via Gradescope

In [ ]:
import scipy.linalg as la
import numpy as np

## Throwback: Solving a Linear System by Hand

We want to solve the following system:

$$3 x_1 + 2 x_2 + x_3 = 6$$
$$-x_1 + 4 x_2 + 5x_3 = 8$$
$$2x_1 -8 x_2 + 10 x_3 = 4$$

In matrix form this looks like

$$\underbrace{\begin{pmatrix} 3 & 2 & 1\\ -1 & 4 & 5\\ 2& -8 & 10\end{pmatrix}}_{\mathbf{A}} ~
\underbrace{\begin{pmatrix} x_1\\x_2\\x_3\end{pmatrix}}_{\mathbf{x}} = \underbrace{\begin{pmatrix}6\\8\\4\end{pmatrix}}_{\mathbf{b}}
$$

Another way to write this is a notational shorthand called an **augmented matrix**, where we put the righthand side into the matrix:

$$
\left(\begin{array}{ccc|c}   3 & 2 & 1 &6\\ -1 & 4 & 5&8\\ 2& -8 & 10&4\end{array}\right)
$$

Now let's apply Gaussian elimination to solve this system of equations with pencil and paper.

## Solving a Linear System with Python

Now let's solve this system of equations with Python. Tip: `scipy` is a little more sophisticated than `numpy`. Read more [here](https://docs.scipy.org/doc/scipy/tutorial/linalg.html#).

In [ ]:
A = np.array([[3, 2, 1], [-1, 4, 5], [2, -8, 10]])
print("A=\n",A)

# Define b
### BEGIN SOLUTION
b = np.array([6,8,4])
### END SOLUTION
print("b=\n",b)

# Calculate x using la.solve
### BEGIN SOLUTION
x = la.solve(A,b)
### END SOLUTION
print("x=\n",x)



We can also use Python to calculate the reduced row echelon form. First, let's use `sympy`, which Savov uses too.

In [ ]:
from sympy import Matrix

# Define A
A_ = Matrix([[3, 2, 1], [-1, 4, 5], [2, -8, 10]])
print("A=\n",A_)

# Define B
b_ = Matrix([6,8,4])
print("b=\n",b_)

# Create augmented matrix
aug_ = A_.row_join(b_)
print("aug=\n",aug_)

# Calculate reduced row eschelon form
rref_ = aug_.rref()
print(rref_)

`Sympy` performs symbolic computations which are great for teaching but often too slow for numerically solving large problems (e.g., compuational fluid dynamics simulation). We'll focus on using `scipy` for most of this class, although you should be comfortable converting the reduced row echelon form of the augmented matrix back into the solution of the linear system.

## Extending Gaussian Elimination to Invert a Matrix

Now let's "open the black box" to see how `scipy`/`numpy` numerically calculates the inverse. Start by studying [](../04/Gauss-Elimination.ipynb) which converts our by-hand Gaussian elimination calculation into pseudocode and then Python code. 

As discussed on pg. 199 of Savov, we can calculate the inverse of a matrix $\bf A$ by assembling the augemented matrix

$$ \begin{bmatrix} {\bf A} & {\bf I} \end{bmatrix} $$

and applying Gauss-Jordan elimination to obtain:

$$ \begin{bmatrix} {\bf I} & {\bf A}^{-1} \end{bmatrix}$$

**Assignment**: 
1. Modify the `GaussElim` and `BackSub` functions from [](../04/Gauss-Elimination.ipynb) to perform Gauss-Jordan elimination. As written, these functions first perform Gaussian elimination to produce zeros below the diagonal in the augmented matrix. Next, a backsolve is performed to solve the linear system. This needs to be modified to instead converted the augmented matrix into reduced row eschelon form, i.e., the left of the augmented matrix should be the identify matrix.
2. Test your function by solving the linear system example ${\bf A x} = {\bf b}$ used earlier in this notebook. In other words, you should be able to reproduce the RREF calculated above with `sympy`.
2. Use your function to calculate the matrix inverse ${\bf A}^{-1}$.
3. Tip: You do not need to worry about pivoting. While pivoting is important for numeric stability, it adds complexity.

In [ ]:
def reduced_row_eschelon_form(aug,LOUD=True):
    """Perform Gauss-Jordan elimination to convert
    an input augmented matrix into reduced row eschelon form

    Args:
        aug: augmented matrix with dimensions N x M, M >= N
    Returns:
        augmented matrix in reduced row eschelon form
    """
    # Extract dimensions of A
    [Nrow, Ncol] = aug.shape
    
    # Check the dimensions
    assert Nrow <= Ncol, "Augmented matrix has too few columns"

    # Create a copy of the augemented matrix
    aug_matrix = aug.copy()
    
    # loop over the first Nrow columns
    for column in range(0,Nrow):
        
        # loop over the rows entries to the right of the pivot
        for row in range(column+1, Nrow):

            ### BEGIN SOLUTION

            # select row to modify
            mod_row = aug_matrix[row,:]

            # modify the entire row
            mod_row -= (mod_row[column]/aug_matrix[column,column]*
                        aug_matrix[column,:])
            
            # overwrite row. Is this needed? Why or why not?
            aug_matrix[row] = mod_row

            ### END SOLUTION

        # end loop over rows
    # end loop over columns

    if LOUD:
        print("\nEliminated below the diagonal:\n",aug_matrix)

    # check augmented matrix is all zeros below the diagonal
    z_tol = 1E-8
    for r in range(Nrow):
        for c in range(0,r):
            assert np.abs(aug_matrix[r,c]) < z_tol, "\nWarning! Nonzero in position "+str(r)+","+str(c)

    # loop over the columns starting at the right (count backwards)
    for column in range(Nrow-1, -1, -1):

        # loop over the rows above the pivot column
        for row in range(0, column):

            ### BEGIN SOLUTION

            # select row to modify
            mod_row = aug_matrix[row,:]

            # modify the entire row
            mod_row -= (mod_row[column]/aug_matrix[column,column]*
                        aug_matrix[column,:])
            
            # overwrite row. Is this needed? Why or why not?
            aug_matrix[row] = mod_row

            ### END SOLUTION

        # end loop over rows
    # end loop over columns

    if LOUD:
        print("\nEliminated above the diagonal:\n",aug_matrix)

    # rescale each row
    for row in range(0,Nrow):
        aug_matrix[row,:] = aug_matrix[row,:] / aug_matrix[row,row]

    if LOUD:
        print("\nReduced row eschelon form:\n",aug_matrix)

    return aug_matrix



# Create the augmented matrix and convert elements into floats (instead of ints)
aug = np.column_stack((A,b)) * 1.0
print("Augmented Matrix:\n",aug)

rref = reduced_row_eschelon_form(aug,LOUD=True)

## Understanding the Determinant